In [ ]:
! pip install rasterio tensorflow scikit-learn tqdm

import os
import numpy as np
import rasterio
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
# Define month-year keys
time_keys = ['2022_Jul', '2023_Jan', '2023_Jul', '2024_Jan', '2024_Jul', '2025_Jan']

# Define paths for each time point
file_paths = {
    '2022_Jul': {
        'pred': r"C:\Users\DELL\OneDrive\Desktop\Documents\BTP\Results\Results_July_2022\predicted_ndwi_July_2022_normalized.tif",
        'act':  r"C:\Users\DELL\OneDrive\Desktop\Documents\BTP\Results\Results_July_2022\ndwi_2022_07.tif"
    },
    '2023_Jan': {
        'pred': r"C:\Users\DELL\OneDrive\Desktop\Documents\BTP\Results\Results_Jan_2023\predicted_ndwi_Jan_2023_normalized.tif",
        'act':  r"C:\Users\DELL\OneDrive\Desktop\Documents\BTP\Results\Results_Jan_2023\ndwi_2023_01.tif"
    },
    '2023_Jul': {
        'pred': r"C:\Users\DELL\OneDrive\Desktop\Documents\BTP\Results\Results_July_2023\predicted_ndwi_July_2023_normalized.tif",
        'act':  r"C:\Users\DELL\OneDrive\Desktop\Documents\BTP\Results\Results_July_2023\ndwi_2023_07.tif"
    },
    '2024_Jan': {
        'pred': r"C:\Users\DELL\OneDrive\Desktop\Documents\BTP\Results\Results_Jan_2024\predicted_ndwi_Jan_2024_normalized.tif",
        'act':  r"C:\Users\DELL\OneDrive\Desktop\Documents\BTP\Results\Results_Jan_2024\ndwi_2024_01.tif"
    },
    '2024_Jul': {
        'pred': r"C:\Users\DELL\OneDrive\Desktop\Documents\BTP\Results\Results_July_2024\predicted_ndwi_July_2024_normalized.tif",
        'act':  r"C:\Users\DELL\OneDrive\Desktop\Documents\BTP\Results\Results_July_2024\ndwi_2024_07.tif"
    },
    '2025_Jan': {
        'pred': r"C:\Users\DELL\OneDrive\Desktop\Documents\BTP\Results\Results_Jan_2025\predicted_ndwi_Jan_2025_normalized.tif",
        'act':  r"C:\Users\DELL\OneDrive\Desktop\Documents\BTP\Results\Results_Jan_2025\ndwi_2025_01.tif"
    }
}

# Define error threshold (in NDWI units)
threshold = 0.1

results = []

for key in time_keys:
    with rasterio.open(file_paths[key]['pred']) as src_pred:
        pred = src_pred.read(1)
    with rasterio.open(file_paths[key]['act']) as src_act:
        act = src_act.read(1)

    # Mask valid pixels
    mask = ~np.isnan(pred) & ~np.isnan(act)
    y_true = act[mask].flatten()
    y_pred = pred[mask].flatten()

    # Threshold-based accuracy
    abs_error = np.abs(y_pred - y_true)
    accurate_pixels = abs_error < threshold
    pixel_accuracy = np.sum(accurate_pixels) / len(y_true)

    # Store results
    year, month = key.split('_')
    results.append({
        'Year': year,
        'Month': month,
        'Accuracy': pixel_accuracy
    })

# Convert to DataFrame
df = pd.DataFrame(results)
table_acc = df.pivot(index='Year', columns='Month', values='Accuracy')
print(f"\nAccuracy Table for ndwi(Threshold = {threshold}):\n", table_acc.round(4))



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Accuracy Table for TCI(Threshold = 0.1):
 Month     Jan     Jul
Year                 
2022      NaN  0.8104
2023   0.7081  0.1114
2024   0.5539  0.6777
2025   0.0535     NaN
